This notebook gives an example on experiment data analysis. 


## Table of Contents

- [1. Imports & Model Setup](#Imports-&-Model-Setup)  
- [2. Single‑File Check](#Single‑File-Check)  
- [3. Multi‑File Aggregation](#Multi‑File-Aggregation)  

# Imports-&-Model-Setup

In [ ]:
# Import the needed package 
import numpy as np 
import torch
import os
import glob
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

from utils.data_loader import expdata_preprocess, data_loader_exp, load_file
from utils.post_analysis import re_group_exp, find_particle_locations
from model import unet_locD

In [ ]:
# 1. Define the path


dir_exp = 'Z:/Dongyu Fan/2. Data/4D data/02192025/crop/'  # Set this as where your data are. 


# Set up the model: 
dir_model = 'C:/Users/dongyuf2/OneDrive - University of Illinois - Urbana/shareddrive/2. Data/ImageProcessing/Training/2025-02-14/02-14_00-16/model_checkpoint_3.pth'  # Set this at where your model is 
D_range = [0.01, 2]

In [ ]:
# Load the model 
# Initialize the model 
model = unet_locD(n_channels=3, n_classes=1,bilinear=False)
# Load the saved state_dict
model.load_state_dict(torch.load(dir_model,map_location=torch.device('cpu') ))
# Set to evaluation mode
model.eval()

# 3.  Perform evaluation
# Set up the device: 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Some parameter for cropping
patchsize = 64 # The spatial size for every patch. Match that of the model input.
overlap = 0.2  # 20% of overlap between patches.
stacksize = 3  # The number of frames to take in every time.

# Single‑File-Check

In [ ]:
# Load the files

# load the data and break into small pieces and save 

imgfile = 'gbeads_100nm_30gly_2D_00_crop.tif'

# create a subfolder for saving first: 
crop_save = dir_exp +  imgfile[:-4] + '/'

# Create directories
os.makedirs(crop_save, exist_ok=True)  # exist_ok=True to avoid error if the directory already exists

expdata_preprocess(dir_exp + imgfile, crop_save, patchsize, overlap, stacksize)



In [ ]:
# Check the loaded data:

dataset = data_loader_exp(crop_save)
#loader_args = dict(batch_size=1, num_workers=os.cpu_count(), pin_memory=True) # Multi processing introduces errors
exp_data = DataLoader(dataset,batch_size=1)

for batch in exp_data:
    a = (batch['image'])
    ind = batch['index']
    a = a.squeeze(0)   # Reduce the dimension for batch. 
    #print(a.max())
    
    plt.imshow(a[0,:,:])
    plt.show()
    break 

# Plot an example frame from a crop 
#plt.imshow(a[0,:,:])

In [ ]:
#  Input into the model 
# send the model to device
model = model.to(device)
# set the path for result saving 
result_save = crop_save + 'result/'
# create the path if needed 
os.makedirs(result_save, exist_ok=True)  # exist_ok=True to avoid error if the directory already exists



In [ ]:
# Start the evaluation 
for batch in exp_data:
    images = batch['image']
    #images = images.squeeze(0)   # Reduce the dimension for batch. 
    #print(images.max())
    images = images.to(device)
    
    index = batch['index']
    # Forward pass
    pred = model(images)  # Ensure you unpack the outputs
    #pred = torch.sigmoid(pred).squeeze().detach().numpy()
    pred = torch.sigmoid(pred).to(dtype=torch.float32).squeeze().permute(1,2,0).detach().cpu().numpy()
    np.savez(result_save + 'result_' + str(index.detach().numpy()[0]) + '.npz', result = pred)
    #plt.imshow(np.sum(pred,axis = 2))
    #plt.show()

    #print(find_particle_locations(pred))
    
    #break
    


In [ ]:
# Regroup the output small patches: 
reconstructed_result = re_group_exp(result_save, 64,64,1000, patchsize,overlap,stacksize)

In [ ]:
# Take a look of the output 
plt.imshow(np.sum(reconstructed_result[:,:,:,100],axis = 2),cmap = 'gray')

In [ ]:
original_img = load_file(dir_exp + imgfile)
# Take a look of the original image for comparison
plt.imshow(original_img[300,:,:] , cmap = 'gray')

In [ ]:
# Do fittings on the recovered results. 
# Parameters
neighborhood_size = 1  # Adjust for PSF size
threshold = 0.25  # Adjust based on noise level! This is the main parameter to adjust here.
location_all = []
for stack in range(reconstructed_result.shape[3]):
    particle_locations = find_particle_locations(reconstructed_result[:,:,:,stack], neighborhood_size, threshold)
    location_all.append(particle_locations)
    #break


In [ ]:
# Check how the recovered results overlay on the original result 

stack = 0


for frame in range(stacksize):
    fig, ax = plt.subplots()
    ax.imshow(original_img[stack * 3 + frame,:,:] , cmap = 'gray')
    #ax.imshow(original_img[200 * 3 + frame,:,:] , cmap = 'gray')
    for pair in location_all[stack]:
        x,y,d = pair
        plt.scatter(x,y,marker = 'o', edgecolors= 'red', s = 100, facecolor = 'None' )
        D = ((d - 2) / 9 ) * (D_range[1] - D_range[0]) + D_range[0] 
        plt.text( x  -2, y -2, f'{D:.2f}', color = 'red')

    # Remove axis labels and ticks
    plt.axis('off')

    # Add a scale bar

    scalebar = ScaleBar(0.1, 'um', location = 'lower right', box_alpha = 0.5)
    ax.add_artist(scalebar)



In [ ]:
# Save the plotted results s a movie 

plots_save = crop_save + 'plots/retrained_model/'
os.makedirs(plots_save, exist_ok=True)

for stack in range(np.shape(reconstructed_result)[3]):
#for stack in range(100):
    for frame in range(stacksize):
        fig, ax = plt.subplots()
        ax.imshow(original_img[stack * 3 + frame,:,:] , cmap = 'gray')
        for pair in location_all[stack]:
            x,y,d = pair
            D = ((d - 2) / 9 ) * (D_range[1] - D_range[0]) + D_range[0] 
            #if D > 1.75:
            #    continue 
            plt.scatter(x,y,marker = 'o', edgecolors= 'red', s = 100, facecolor = 'None' )

            plt.text( x- 2, y - 2, f'{D:.2f}', color = 'red')

        # Remove axis labels and ticks
        plt.axis('off')

        # Add a scale bar

        scalebar = ScaleBar(0.1, 'um', location = 'lower right', box_alpha = 0.5)
        ax.add_artist(scalebar)

        plt.savefig(plots_save + f'Plots_{stack*3 + frame}.png')
        plt.show()


In [ ]:
# Check the output movie to see if identifications are accurate

In [ ]:
# Save the histogram data for plotting in the matlab 
import scipy
D_array = np.array(D_all)
os.makedirs(crop_save + 'output/', exist_ok = True)
scipy.io.savemat(crop_save + 'output/D.mat',{'D_array' : D_array})


In [ ]:
# 8. List a map 
# Let's do this whole thing for all the data:

# Start the evaluation: 
# Initiate the arrays 
canvas = np.zeros((64, 64), dtype=float)
count = np.zeros((64, 64), dtype=int)


for frame in location_all:
    for particle in frame:      
        x =  particle[0].astype(int)
        y = particle[1].astype(int)
        values = ((particle[2] - 2) / 9.0) * (D_range[1] - D_range[0]) + D_range[0]
        # Accumulate sums and counts using np.add.at
        if values < 1.7:
            np.add.at(canvas, (y, x), values)  
            np.add.at(count, (y, x), 1)


# Take care of the post process:

canvas_all = np.zeros_like(canvas)
np.divide(canvas, count, out=canvas_all, where=(count != 0))


In [ ]:
# Plot the diffusion map:

plt.imshow(canvas_all,cmap = 'jet')
plt.colorbar()
plt.axis('off')

# Multi‑File-Aggregation

In [ ]:
# reads files: 

file_stem = 'roi_agrose08_fov2'   # The name of the file starts with 
files = glob.glob(os.path.join(dir_exp,file_stem+'*.tif'))


In [ ]:
# Initiate some files
location_all_comb = []


In [ ]:
# Loop over all the files 
for file in files:
    

    # create a subfolder for saving first: 
    crop_save = file[:-4] + '/'

    # Create directories
    os.makedirs(crop_save, exist_ok=True)  # exist_ok=True to avoid error if the directory already exists

    expdata_preprocess(file, crop_save, patchsize, overlap, stacksize)


    # Setup the location for saving results: 
    result_save = crop_save + 'result/'
    # create the path if needed 
    os.makedirs(result_save, exist_ok=True)  # exist_ok=True to avoid error if the directory already exists


    # Load data:

    
    dataset = data_loader_exp(crop_save)
    #loader_args = dict(batch_size=1, num_workers=os.cpu_count(), pin_memory=True) # Multi processing introduces errors
    exp_data = DataLoader(dataset,batch_size=1)
    # Then read and pred 
    # Start the evaluation 
    
    for batch in exp_data:
        images = batch['image']
        images = images.to(device)
        
        index = batch['index']
        # Forward pass
        pred = model(images)  # Ensure you unpack the outputs
        pred = torch.sigmoid(pred).to(dtype=torch.float32).squeeze().permute(1,2,0).detach().cpu().numpy()
        np.savez(result_save + 'result_' + str(index.detach().numpy()[0]) + '.npz', result = pred)
    
    # Then get the locations:
    # Regroup the output small patches: 
    reconstructed_result = re_group_exp(result_save, 64,64,2000, patchsize,overlap,stacksize)

    # Do fittings on the recovered results. 
    # Parameters
    neighborhood_size = 1  # Adjust for PSF size
    threshold = 0.4  # Adjust based on noise level
    for stack in range(reconstructed_result.shape[3]):
        particle_locations = find_particle_locations(reconstructed_result[:,:,:,stack], neighborhood_size, threshold)
        location_all_comb.extend(particle_locations)
        #break
    
    print("Finished Process " + file)
    
    

In [ ]:
# Let's do this whole thing for all data in the same FOV

# Start the evaluation: 
# Initiate the arrays 
canvas = np.zeros((64, 64), dtype=float)
count = np.zeros((64, 64), dtype=int)

D_all = []
for particle in location_all_comb:
     
    x =  particle[0].astype(int)
    y = particle[1].astype(int)
    values = ((particle[2] - 2) / 9.0) * (D_range[1] - D_range[0]) + D_range[0]
    # Accumulate sums and counts using np.add.at
    #if values != error_value:
    if values < 2.1:
        np.add.at(canvas, (y, x), values)  
        np.add.at(count, (y, x), 1)
        D_all.append(values)


# Take care of the post process:

canvas_all = np.zeros_like(canvas)
np.divide(canvas, count, out=canvas_all, where=(count > 4) )


In [ ]:
# Save if needed: 
#np.savez(dir_exp + 'fov2.npz', D_all=D_all, location_all = location_all_comb)

# Import if needed:
#data = np.load(dir_exp + 'fov3.npz' )
#D_all = data['D_all']
#location_all_comb = data['location_all']


In [ ]:
# Save matlab if needed.
import scipy
#scipy.io.savemat(dir_exp + 'fov3.mat', {'D_all': D_all, 'location_all':location_all_comb} )

In [ ]:
# Check out the Diffusion map 
plt.imshow(canvas_all,cmap = 'jet')
plt.colorbar()
plt.axis('off')